In [12]:
print("Hello Jupyter")

Hello Jupyter


In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

In [15]:
import requests
import json

url = "https://api.openai.com/v1/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai_api_key
}
data = {
    "model": "gpt-3.5-turbo",
    "prompt": "hello!",
    "temperature": 0
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

{
  "error": {
    "message": "This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?",
    "type": "invalid_request_error",
    "param": "model",
    "code": null
  }
}


In [16]:
"""
import requests
import json

url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + openai_api_key
}
data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role" : "user", "content": "hello!"}
    ],
    "temperature": 0
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))
"""

'\nimport requests\nimport json\n\nurl = "https://api.openai.com/v1/chat/completions"\nheaders = {\n    "Content-Type": "application/json",\n    "Authorization": "Bearer " + openai_api_key\n}\ndata = {\n    "model": "gpt-3.5-turbo",\n    "messages": [\n        {"role" : "user", "content": "hello!"}\n    ],\n    "temperature": 0\n}\n\nresponse = requests.post(url=url, headers=headers, json=data)\nprint(json.dumps(response.json(), indent=2))\n'

# Section 4: LangChain入門（前半）

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
result = llm.invoke("自己紹介してください。")
print(result)

content='こんにちは！私はAIアシスタントのChatGPTです。さまざまな質問に答えたり、情報を提供したり、会話を楽しんだりするために設計されています。趣味や興味、学びたいことについてお話しするのが好きです。何か特定のことについてお話ししたいことがあれば、ぜひ教えてください！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 11, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-a6c6f3c6-b989-4f11-8fc4-dacac5ea00e3-0' usage_metadata={'input_tokens': 11, 'output_tokens': 87, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [18]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template
)

result = prompt.format(command="ls")
print(result)


次のコマンドの概要を説明してください。

コマンド: ls



In [19]:
import langchain
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template
)

chain = LLMChain(llm=chat, prompt=prompt)
result = chain.run("ls")
print(result)



> Entering new LLMChain chain...
Prompt after formatting:

次のコマンドの概要を説明してください。

コマンド: ls



/tmp/ipykernel_3429/1441021757.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=prompt)



> Finished chain.
`ls` コマンドは、Unix系のオペレーティングシステム（LinuxやmacOSなど）で使用されるコマンドで、指定したディレクトリ内のファイルやサブディレクトリの一覧を表示するためのものです。デフォルトでは、現在の作業ディレクトリの内容を表示します。

主な機能やオプションには以下のようなものがあります：

- `-l` オプション: 詳細表示モードで、ファイルのパーミッション、所有者、サイズ、最終更新日時などの情報を含むリストを表示します。
- `-a` オプション: 隠しファイル（名前がドットで始まるファイル）も含めてすべてのファイルを表示します。
- `-h` オプション: サイズを人間に読みやすい形式（KB, MBなど）で表示します。
- `-R` オプション: サブディレクトリも再帰的に表示します。

このように、`ls` コマンドはファイルシステムの内容を確認するための基本的かつ非常に便利なツールです。


In [20]:
cot_template = """
以下の質問に回答してください。

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template
)
cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

summarize_template = """
入力を結論だけ一言に要約してください。

### 入力 ###
{input}
### 入力終了 ###
"""

summarize_prompt = PromptTemplate(
    input_variables=["inupt"],
    template=summarize_template
)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [21]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain("私は市場に行って１０個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？")
print(result["output"])



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

以下の質問に回答してください。

### 質問 ###
私は市場に行って１０個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？
### 質問終了 ###

ステップバイステップで考えましょう。



/tmp/ipykernel_3429/3858086507.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = cot_summarize_chain("私は市場に行って１０個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？")



> Finished chain.
では、ステップバイステップで考えてみましょう。

1. **最初に買ったリンゴの数**: 10個
2. **隣人に渡したリンゴの数**: 2個
3. **修理工に渡したリンゴの数**: 2個

   ここで、隣人と修理工に渡したリンゴの合計は:
   \[
   2 + 2 = 4 \text{個}
   \]

4. **残りのリンゴの数**: 
   最初に買ったリンゴから渡したリンゴを引きます。
   \[
   10 - 4 = 6 \text{個}
   \]

5. **さらにリンゴを買った数**: 5個
6. **新しい合計のリンゴの数**:
   残りのリンゴに新たに買ったリンゴを加えます。
   \[
   6 + 5 = 11 \text{個}
   \]

7. **食べたリンゴの数**: 1個
8. **最終的な残りのリンゴの数**:
   新しい合計から食べたリンゴを引きます。
   \[
   11 - 1 = 10 \text{個}
   \]

したがって、残りのリンゴは **10個** です。


> Entering new LLMChain chain...
Prompt after formatting:

入力を結論だけ一言に要約してください。

### 入力 ###
では、ステップバイステップで考えてみましょう。

1. **最初に買ったリンゴの数**: 10個
2. **隣人に渡したリンゴの数**: 2個
3. **修理工に渡したリンゴの数**: 2個

   ここで、隣人と修理工に渡したリンゴの合計は:
   \[
   2 + 2 = 4 \text{個}
   \]

4. **残りのリンゴの数**: 
   最初に買ったリンゴから渡したリンゴを引きます。
   \[
   10 - 4 = 6 \text{個}
   \]

5. **さらにリンゴを買った数**: 5個
6. **新しい合計のリンゴの数**:
   残りのリンゴに新たに買ったリンゴを加えます。
   \[
   6 + 5 = 11 \text{個}
   \]

7. **食べたリンゴの数**: 1個
8. **最終的な残りのリンゴの数**:
   新しい合計から食べたリンゴを引きます。
   \[
   

In [4]:
from langchain.globals import set_verbose
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from typing import List

set_verbose(True)
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="ingredients of the dish")
    steps: List[str] = Field(description="steps to make the dish")

template = """料理のレシピを教えてください。
{format_instructions}

料理名: {dish}
"""

parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = LLMChain(llm=chat, prompt=prompt)
output = chain.run(dish="カレー")
print(output)

/tmp/ipykernel_974/918305817.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=prompt)
/tmp/ipykernel_974/918305817.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run(dish="カレー")




> Entering new LLMChain chain...
Prompt after formatting:
料理のレシピを教えてください。
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```

料理名: カレー


> Finished chain.
```json
{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
  

In [17]:
recipe = parser.parse(output)
print(recipe)
print(type(recipe))

ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体を混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮る。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']
<class '__main__.Recipe'>


# Section 5: lnagChain入門（後半）

In [3]:
"""
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator

loader = DirectoryLoader("../langchain/docs", glob="**/*.md")
embedding = OpenAIEmbeddings()
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
"""

'\nfrom langchain_community.document_loaders import DirectoryLoader\nfrom langchain_community.embeddings import OpenAIEmbeddings\nfrom langchain.indexes import VectorstoreIndexCreator\n\nloader = DirectoryLoader("../langchain/docs", glob="**/*.md")\nembedding = OpenAIEmbeddings()\nindex = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])\n'

In [1]:
"""
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4o-mini")
result = index.query("LangChainの概要を一文で説明してください", llm=chat)
print(result)
"""

NameError: name 'index' is not defined

In [22]:
from dotenv import load_dotenv
import requests
import json
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

def post_chat_completions(content):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai_api_key
    }
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role" : "user", "content": content}
        ],
        "temperature": 0
    }

    response = requests.post(url=url, headers=headers, json=data)
    print(json.dumps(response.json(), indent=2))

In [23]:
post_chat_completions("Hi. I'm the eminence in shadow.")

{
  "id": "chatcmpl-AnnfZ8pXHcIyWOKly9VAWNk4e7xAZ",
  "object": "chat.completion",
  "created": 1736432365,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello, Eminence in Shadow. How can I assist you today?",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 18,
    "completion_tokens": 15,
    "total_tokens": 33,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "system_fingerprint": null
}


In [24]:
post_chat_completions("Do you know my name?")

{
  "id": "chatcmpl-AnnfackHZybY160USgjKGvUaDx00d",
  "object": "chat.completion",
  "created": 1736432366,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I'm sorry, I do not know your name.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 12,
    "total_tokens": 25,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "system_fingerprint": null
}


In [25]:
post_chat_completions(
    """
    Human: Hi. I'm the eminence in shadow.
    AI: Hello! How can I assist you today?
    Humna: Do you know my name?
    AI: 
    """
)

{
  "id": "chatcmpl-AnnfbuDFg9XCCpttKk9eF7vq0pU0c",
  "object": "chat.completion",
  "created": 1736432367,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, you just introduced yourself as the eminence in shadow.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 49,
    "completion_tokens": 15,
    "total_tokens": 64,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "system_fingerprint": null
}


In [23]:
"""
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

#langchain.verbose = True
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=100)
conversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())

while True:
    user_message = input("You: ")
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")
"""
    

'\nfrom langchain.chains import ConversationChain\nfrom langchain.chat_models import ChatOpenAI\nfrom langchain.memory import ConversationBufferMemory\n\n#langchain.verbose = True\nchat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=100)\nconversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())\n\nwhile True:\n    user_message = input("You: ")\n    ai_message = conversation.predict(input=user_message)\n    print(f"AI: {ai_message}")\n'

In [10]:
"""
from langchain.globals import set_verbose
from langchain.agents import load_tools
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

set_verbose(True)

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

tools = load_tools(["terminal"], allow_dangerous_tools=True, llm=chat)
agent_chain = initialize_agent(tools, chat, agent="zero-shot-react-description")

result = agent_chain.run("What is your current directory?")
print(result)
"""

'\nfrom langchain.globals import set_verbose\nfrom langchain.agents import load_tools\nfrom langchain.agents import load_tools\nfrom langchain.agents import initialize_agent\nfrom langchain.chat_models import ChatOpenAI\n\nset_verbose(True)\n\nchat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)\n\ntools = load_tools(["terminal"], allow_dangerous_tools=True, llm=chat)\nagent_chain = initialize_agent(tools, chat, agent="zero-shot-react-description")\n\nresult = agent_chain.run("What is your current directory?")\nprint(result)\n'

In [4]:
"""
from langchain.globals import set_verbose
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import openai

set_verbose(True)
openai.log = "debug"
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=100)
conversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())

while True:
    user_message = input("You: ")
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")
"""

'\nfrom langchain.globals import set_verbose\nfrom langchain.chains import ConversationChain\nfrom langchain.chat_models import ChatOpenAI\nfrom langchain.memory import ConversationBufferMemory\nimport openai\n\nset_verbose(True)\nopenai.log = "debug"\nchat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=100)\nconversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())\n\nwhile True:\n    user_message = input("You: ")\n    ai_message = conversation.predict(input=user_message)\n    print(f"AI: {ai_message}")\n'

In [1]:
from langchain.globals import set_verbose
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

set_verbose(True)
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=100)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi I'm Oshima!")
]

result = chat(messages)
print(result)

/tmp/ipykernel_5013/1731157905.py:13: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chat(messages)


content='Hi Oshima! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 22, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-4da3d339-0591-4316-9316-63218c32a770-0' usage_metadata={'input_tokens': 22, 'output_tokens': 12, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
"""
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

while True:
    user_message = input("You: ")
    memory.chat_memory.add_user_message(user_message)

    ai_message = chat(memory.chat_memory.messages)
    memory.chat_memory.add_ai_message(ai_message.content)
    print(f"AI: {ai_message.content}")
"""

'\nfrom langchain.memory import ConversationBufferMemory\nmemory = ConversationBufferMemory()\n\nwhile True:\n    user_message = input("You: ")\n    memory.chat_memory.add_user_message(user_message)\n\n    ai_message = chat(memory.chat_memory.messages)\n    memory.chat_memory.add_ai_message(ai_message.content)\n    print(f"AI: {ai_message.content}")\n'